In [ ]:
import pandas as pd, numpy as np
import json
from tqdm import tqdm

def process(input_str):
    return json.loads(input_str)

def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]

def load_json(data):
    data.loc[:, 'prompt'] = data['prompt'].apply(process)
    data.loc[:, 'response_a'] = data['response_a'].apply(process)
    data.loc[:, 'response_b'] = data['response_b'].apply(process)
    return data

In [ ]:
data = pd.read_csv("dataset/train.csv")

In [ ]:
data = load_json(data)

In [ ]:
data = data.explode(['prompt','response_a','response_b']).reset_index(drop = True)

In [ ]:
data['label'] = data.apply(lambda x: get_label(x), axis = 1)

In [ ]:
data = data.fillna('None')

In [ ]:
data['response_a'] = data['response_a'].apply(lambda x: 'None' if len(x)==0 else x)
data['response_b'] = data['response_b'].apply(lambda x: 'None' if len(x)==0 else x)

In [ ]:
#prompt1
'''
#Model A
Prompt1: xxx
Response: xxx

Prompt2: xxx
Response: xxx

#Model B
Prompt1: xxx
Response: xxx

Prompt2: xxx
Response: xxx
'''

In [ ]:
data['prompt_response_A'] = "Prompt: " + data['prompt'] + "\n" + "Response: " + data['response_a']
data['prompt_response_B'] = "Prompt: " + data['prompt'] + "\n" + "Response: " + data['response_b']
data = data.groupby('id').agg({'prompt_response_A': '\n\n'.join, 'prompt_response_B': '\n\n'.join, 'label': lambda x: list(x)[0]}).reset_index()
data['prompt_response'] = "#Model A\n" + data['prompt_response_A'] + "\n\n#Model B\n" + data['prompt_response_B']

In [ ]:
print(data.prompt_response.values[0])

In [ ]:
#prompt2
'''
Label A
#prompt1
xxxx
#Response
##Model A
##Model B

到这里超过max length了，另外变成一行，Label还是A

#Prompt2
xxxxx
#Response
##Model A
xxxx
##Model B
xxxx
'''


In [ ]:
data['prompt_response'] = "#Prompt\n" + data['prompt'] + "\n\n" + "#Response\n" + "##Model A\n" + data['response_a'] + "\n\n" + "##Model B\n" + data['response_b']

In [ ]:
max_length = 3000

In [ ]:
prompt_response = []
ids = []
labels = []
text_length = 0
for idx, row in tqdm(data.iterrows(), total=len(data)):
    text = row['prompt_response']
    label = row['label']
    id = row['id']
    if id not in ids:
        #第一次出现
        prompt_response.append(text)
        text_length = len(text.split(" "))
        ids.append(id)
        labels.append(label)
    else:
        text_length += len(text.split(" "))
        if text_length <= max_length:
            #print(f"before: {prompt_response[-1]}")
            #取上一个text出来，合并后替换
            text = prompt_response[-1] + "\n\n" + text
            prompt_response[-1] = text
            #print(f"after: {prompt_response[-1]}")
        else:
            #另一起一行
            prompt_response.append(text)
            text_length = len(text.split(" "))
            ids.append(id)
            labels.append(label)
    

In [ ]:
data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response, "labels": labels})

In [ ]:
data.id.value_counts()

In [ ]:
tmp = data.loc[data.id == 2995198009,:]
tmp

In [ ]:
print(tmp.response_b.values[-2])

In [ ]:
print(tmp.prompt_response.values[-1])